In [1]:
import sys
import random
import numpy as np
import tensorflow as tf

In [3]:
# Load the sonnet text file and convert to lowercase
sonnet_text = open("./data/Sonnet.txt", "r").read().lower()

# Create a set of unique characters present in the text
chars = sorted(list(set(sonnet_text)))

In [5]:
# Create dictionaries to map characters to indices and vice versa
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [6]:
# Create input sequences and output labels by sliding a window of length seq_length
seq_length = 40
step_size = 3
input_seqs = []
output_labels = []
for i in range(0, len(sonnet_text) - seq_length, step_size):
    input_seqs.append(sonnet_text[i:i+seq_length])
    output_labels.append(sonnet_text[i+seq_length])

In [7]:
# Convert input sequences and output labels to arrays of indices
X = np.zeros((len(input_seqs), seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(input_seqs), len(chars)), dtype=np.bool)
for i, input_seq in enumerate(input_seqs):
    for j, char in enumerate(input_seq):
        X[i, j, char_to_idx[char]] = 1
    y[i, char_to_idx[output_labels[i]]] = 1

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9508\2501761822.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_seqs), seq_length, len(chars)), dtype=np.bool)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9508\2501761822.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_seqs), len(chars)), dtype=np.bo

In [8]:
# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(seq_length, len(chars))),
    tf.keras.layers.Dense(len(chars), activation="softmax")
])

In [9]:
# Compile the model with categorical crossentropy loss and RMSprop optimizer
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [20]:
def generated_text():
    # Generate text after each epoch
    start_index = random.randint(0, len(sonnet_text) - seq_length - 1)
    generated_text = sonnet_text[start_index:start_index+seq_length]
    sys.stdout.write(generated_text)
    for i in range(400):
        X_pred = np.zeros((1, seq_length, len(chars)))
        for j, char in enumerate(generated_text):
            X_pred[0, j, char_to_idx[char]] = 1
        pred = model.predict(X_pred, verbose=0)[0]
        next_char = idx_to_char[np.argmax(pred)]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)
        sys.stdout.flush()

In [12]:
# Train the model for 50 epochs
for epoch in range(50):
    print(f"{epoch}/50, going strong!")
    model.fit(X, y, batch_size=128, epochs=1)
    # Generate text after each epoch
    start_index = random.randint(0, len(sonnet_text) - seq_length - 1)
    generated_text = sonnet_text[start_index:start_index+seq_length]
    sys.stdout.write(generated_text)
    for i in range(400):
        X_pred = np.zeros((1, seq_length, len(chars)))
        for j, char in enumerate(generated_text):
            X_pred[0, j, char_to_idx[char]] = 1
        pred = model.predict(X_pred, verbose=0)[0]
        next_char = idx_to_char[np.argmax(pred)]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

0/50, going strong!
249/249 [==============================] - 75s 302ms/step - loss: 0.8443
nothing, sweet boy; but yet, like prayer far me beat,
of the wher art make flom to me warte
and thes my best conger for of the reart,
and thee beauth, bure will though for mestreeps,
and hith paine of asterâ€™d hur thy easty?
when for thy sweet with thee to mide upreess,
and seak showels, and to enem mering mine,
and the gaich that when i head sholl to sees dorngrie.
thes made mine one then, and thereare died;
ithince but till is mant o
1/50, going strong!
249/249 [==============================] - 72s 290ms/step - loss: 0.8134
ove
against strange maladies a sovereign
ain, that for with heroty that dear shertheng stay;
but shemf in mysended bate, and thee in mise:
but she fire, beatthe wordo for the rided;
bet from thy manterâ€™s thy canterange,
and tomy me bateres of the is all he.


as wive wome touts, thou shalt heavâ€™st the hiss sage to crmemm.



ost it of prir: the recell the toll seive,

In [22]:
generated_text()

ng eyelids open wide,
looking on darknes shall you woth thee so bare.
when i ho dour foutind as it pautt wod;
o me but the fair branss my friends wath hard aprithanâ€™s fore,
then my he erssayâ€™d wite she mad mo faill,
alisw in eldee then, eyes as the gands of fould,
the rise comet it anowet hes ade thight
why foul eye when you sofrury beasty,
callow in who hee, they love hesell paise:
sour thou thyself the lasse of my sid wooth,
and m